# Pytorch CUDA Computations

Pytorch supports CUDA tensor types, that implement the same function as CPU
tensors, but utilize GPUs for computation. see https://pytorch.org/docs/stable/cuda.html
for further details.

Important to note is that computation between a CUDA tensor and a CPU tensor is
allowed and with produce an error.

# LTN CUDA impementation

LTN allows for constants, variables, predicates, and functions to be created
directly on CUDA memory, by passing the GPU device as an argument (otherwise defaults to CPU).

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
import logictensornetworks as ltn
import numpy as np

In [2]:
# Check if GPU device is available, and if true store the device name as python variable
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Creating a trainable embedding as a ltn constant on the GPU device (if available)
c1 = ltn.constant(np.random.uniform(low=0.,high=1.,size=4))
c2 = ltn.constant(np.random.uniform(low=0.,high=1.,size=4), device=device)

print('c1\'s device: ', c1.device)
print('c2\'s device: ', c2.device)

c1's device:  cpu
c2's device:  cuda:0


LTN variables can take non-cuda tensors as the feed, but still be assign to the GPU.

*All tensors in the feed must exist on the same device.

In [3]:
c3 = ltn.constant(np.random.uniform(low=0.,high=1.,size=4))
c4 = ltn.constant(np.random.uniform(low=0.,high=1.,size=4), device=device)

v1 = ltn.variable('v1',torch.stack([c1,c3]), device=device) # feed is on CPU, but variable is on GPU
v2 = ltn.variable('v2',torch.stack([c2,c4]), device=device) # feed is on GPU and same for variable
v3 = ltn.variable('v2',torch.stack([c2,c4])) # feed is on GPU, but variable is not

print('v1\'s device: ', v1.device)
print('v2\'s device: ', v2.device)
print('v3\'s device: ', v3.device)

v1's device:  cuda:0
v2's device:  cuda:0
v3's device:  cpu


In [4]:
A = ltn.Predicate.MLP([4,4], hidden_layer_sizes=[8,8])
B = ltn.Predicate.MLP([4,4], hidden_layer_sizes=[8,8], device=device)

print('A\'s device: ', A([c1,c3]).device)
print('B\'s device: ', B([c2,c4]).device)

A's device:  cpu
B's device:  cuda:0


The result of ltn logical operators and qualifiers will result in
tensors on the same device as the input.

In [8]:
and_luk = ltn.fuzzy_ops.And_Luk()

print('AND on cpu tensors :', and_luk(c1,c3).device)
print('AND on gpu tensors :', and_luk(c2,c4).device)

AND on cpu tensors : cpu
AND on gpu tensors : cuda:0
